In [27]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import qutip as qt 
from ECD_control.ECD_optimization.batch_optimizer_new import BatchOptimizer
from ECD_control.gate_sets import ECDGateSet
from ECD_control.ECD_optimization.GateSynthesizer import GateSynthesizer
from ECD_control.ECD_optimization.optimization_analysis import OptimizationAnalysis, OptimizationSweepsAnalysis
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
N = 40
# We initialize the ECDGateSet here
gate_set_params = {
    'N_cav' : N,
    'beta_scale' : 3.0, #maximum |beta| for random initialization  
}
ECD_gate_set = ECDGateSet(**gate_set_params)

In [29]:
#The target oscillator state.
fock = 1
psi_t = qt.basis(N,fock) #target state

In [30]:
#the optimization options
synth_params = {
    'gateset' : ECD_gate_set,
    'N_blocks':6,
    'N_multistart' : 100, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 100, #number of epochs before termination
    'epoch_size' : 10, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 0.995, #terminal fidelitiy
    'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
    'beta_scale' : 3.0, #maximum |beta| for random initialization
    'initial_states' : [qt.tensor(qt.basis(2,0),qt.basis(N,0))], #qubit tensor oscillator, start in |g> |0>
    'target_states' : [qt.tensor(qt.basis(2,1), psi_t)], #end in |e> |target>.
    'name' : 'Fock %d' % fock, #name for printing and saving
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}
gatesynth = GateSynthesizer(**synth_params)

In [31]:
#create optimization object. 
#initial params will be randomized upon creation
opt = BatchOptimizer(gatesynth)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
gatesynth.print_info()

filename: Fock 1.h5

Best circuit parameters found:
betastf.Tensor(
[ 1.09395-1.15969j  1.26394-2.44875j -0.02992+0.0056j  -1.04797+1.81618j
  0.09939+1.32947j  0.63213-0.37139j], shape=(6,), dtype=complex64)
phistf.Tensor([ 0.72704  1.83644 -2.33326  1.74216  2.17263 -0.87265], shape=(6,), dtype=float32)
etastf.Tensor([ 0.14676  2.74993  0.70374 -0.25467 -0.3884   2.14855], shape=(6,), dtype=float32)
thetastf.Tensor([-0.58772 -1.77552 -2.93049  0.69667  2.07901 -1.35963], shape=(6,), dtype=float32)




In [32]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
opt.optimize()

Start time: 2022-02-16 10:08:13


 Optimization stopped. Term fidelity reached.



AttributeError: 'BatchOptimizer' object has no attribute '_save_termination_reason'

In [33]:
#can print info, including the best circuit found.
gatesynth.print_info() 

filename: Fock 1.h5

Best circuit parameters found:
betastf.Tensor(
[-1.17462-0.86936j -0.49395+0.57562j -0.79899+0.0522j   0.16476+0.29228j
 -0.19569-0.38825j  0.03831-0.18385j], shape=(6,), dtype=complex64)
phistf.Tensor([-2.45867  1.31623  3.11642  1.03416  1.11155 -2.95249], shape=(6,), dtype=float32)
etastf.Tensor([ 1.4824   2.50611  2.10898  1.06872  2.3669  -2.27538], shape=(6,), dtype=float32)
thetastf.Tensor([-1.51928  1.41058 -2.41112 -1.96396  2.68694 -0.73564], shape=(6,), dtype=float32)




In [34]:
print(gatesynth.best_fidelity())

0.99766856


In [35]:
#can also get the best circuit parameters directly, could save this to a .npz file.
best_circuit =  gatesynth.best_circuit()
betas = best_circuit['betas']
phis = best_circuit['phis']
thetas = best_circuit['thetas']
print(best_circuit)

{'betas': <tf.Tensor: shape=(6,), dtype=complex64, numpy=
array([-1.1746151 -0.8693623j , -0.49395102+0.5756166j ,
       -0.79899365+0.05219852j,  0.16476296+0.29228175j,
       -0.1956888 -0.38825154j,  0.03830615-0.18385214j], dtype=complex64)>, 'phis': <tf.Tensor: shape=(6,), dtype=float32, numpy=
array([-2.4586654,  1.3162277,  3.1164181,  1.0341647,  1.1115539,
       -2.9524856], dtype=float32)>, 'etas': <tf.Tensor: shape=(6,), dtype=float32, numpy=
array([ 1.4823964,  2.5061128,  2.1089766,  1.0687244,  2.3668997,
       -2.2753816], dtype=float32)>, 'thetas': <tf.Tensor: shape=(6,), dtype=float32, numpy=
array([-1.5192764 ,  1.4105847 , -2.4111152 , -1.9639561 ,  2.6869357 ,
       -0.73564196], dtype=float32)>}
